In [1]:
using Random, CategoricalArrays, DataFrames, DataFramesMeta, StatsBase, CSV, ScikitLearn

In [8]:
data_path_to_load = "C:/Users/15125/Desktop/CAPSTONE PROJECT/CLEANED DATA/FAIRNESS MODEL/"
weighted_sev = CSV.read(data_path_to_load*"weighted_severity_penetrating_0.0001.csv")
orig_label = weighted_sev[!,:label]
updated_label = weighted_sev[!,:updated_label]
IJulia.clear_output()

0

In [29]:
function oct_train(train_X, train_y, test_X, test_y)
    oct = IAI.OptimalTreeClassifier(random_seed=1, criterion=:gini, 
        missingdatamode=:separate_class, treat_unknown_level_missing=true, 
        ls_num_tree_restarts= 60, minbucket= 100)
    grid_oct = IAI.GridSearch(oct,max_depth=10)
    IAI.fit_cv!(grid_oct, train_X, train_y, n_folds=5, sample_weight=:autobalance)
    best_params_grid = IAI.get_best_params(grid_oct)
    best_OCT = IAI.get_learner(grid_oct)
    y_pred_train= IAI.predict(best_OCT, train_X)
    y_pred_test= IAI.predict(best_OCT, test_X)
    train_auc = IAI.score(grid_oct, train_X, train_y, criterion=:auc)
    println(train_auc)
    test_auc = IAI.score(grid_oct, test_X, test_y, criterion=:auc)
    println(test_auc)
    return train_auc, test_auc, best_OCT
end

oct_train (generic function with 1 method)

In [30]:
function pre_process_and_train(X, y, seed_num)
    (train_X, train_y), (test_X, test_y) = IAI.split_data(:classification, X, y, seed= seed_num)
    
    severity_categorical_var = [
    :Head_severity,
    :Face_severity,
    :Neck_severity,
    :Thorax_severity,
    :Abdomen_severity,
    :Spine_severity,
    :Upper_Extremity_severity,
    :Lower_Extremity_severity,
    :Pelvis_Perineum_severity,
    :External_severity,
    :gcstot1,
    #:severity_max
    ]
    
    for col_name in severity_categorical_var
        train_X[!, col_name] = CategoricalArray(train_X[!, col_name], ordered=true)
        test_X[!, col_name] = CategoricalArray(test_X[!, col_name], ordered=true)  
        println("Transformed column '", col_name, "' to Ordered Categorical")
    end
    
    train_X = categorical!(train_X)
    test_X = categorical!(test_X)
    println("Transformed all other non-ordered Categorical Variables")
    
    names = [
        :age, :gender, :signsoflife, :eddisp, :sbp1, :pulse1,
        :oxysat1, :temp1, :gcstot1, :bleeding_disorder, :current_chemotherapy,
        :congestive_heart_failure, :current_smoker, :chronic_renal_failure,
        :history_cva, :diabetes, :disseminated_cancer,  :copd, :steroid,
        :cirrhosis, :history_MI, :history_pvd, :hypertension_medication,
        :method_of_injury, :Head_severity, :Face_severity, :Neck_severity, :Thorax_severity, :Abdomen_severity,
        :Spine_severity, :Upper_Extremity_severity, :Lower_Extremity_severity,
        :Pelvis_Perineum_severity, :External_severity,:alcohol_use_disorder,:drug_use_disorder,:rr1]
    
    train_X = train_X[!, names]
    test_X = test_X[!, names]
    
    train_auc, test_auc, best_OCT = oct_train(train_X, train_y, test_X, test_y)
    
    return  train_auc, test_auc, best_OCT  
end

pre_process_and_train (generic function with 1 method)

## Original Labels

In [31]:
orig_aucs_train = []
orig_aucs_test = []
for num in 1:10
    println("Beginning run number ", num)
    println("\n")
    train_auc, test_auc, best_OCT = pre_process_and_train(weighted_sev, orig_label, num)
    orig_aucs_train = append!(orig_aucs_train, train_auc)
    orig_aucs_test = append!(orig_aucs_test, test_auc)
    IAI.write_json("Fairness_OCT_original_label $num.json",best_OCT)
    println("Ending run number ", num)
    println("\n")
end

Beginning run number 1


Transformed column 'Head_severity' to Ordered Categorical
Transformed column 'Face_severity' to Ordered Categorical
Transformed column 'Neck_severity' to Ordered Categorical
Transformed column 'Thorax_severity' to Ordered Categorical
Transformed column 'Abdomen_severity' to Ordered Categorical
Transformed column 'Spine_severity' to Ordered Categorical
Transformed column 'Upper_Extremity_severity' to Ordered Categorical
Transformed column 'Lower_Extremity_severity' to Ordered Categorical
Transformed column 'Pelvis_Perineum_severity' to Ordered Categorical
Transformed column 'External_severity' to Ordered Categorical
Transformed column 'gcstot1' to Ordered Categorical
Transformed all other non-ordered Categorical Variables


CV fold...                           0%|█               |  ETA: N/A
2mTraining trees...                    0%|█               |  ETA: N/A
2mTraining trees...                    2%|█               |  ETA: 0:10:10
2mTraining trees...                    3%|█               |  ETA: 0:09:46
2mTraining trees...                    5%|█               |  ETA: 0:08:48
2mTraining trees...                    7%|██              |  ETA: 0:07:26
2mTraining trees...                    8%|██              |  ETA: 0:06:31
2mTraining trees...                   10%|██              |  ETA: 0:05:41
2mTraining trees...                   12%|██              |  ETA: 0:05:18
2mTraining trees...                   13%|███             |  ETA: 0:04:57
2mTraining trees...                   15%|███             |  ETA: 0:04:39
2mTraining trees...                   17%|███             |  ETA: 0:04:21
2mTraining trees...                   18%|███             |  ETA: 0:04:08
2mTraining trees...                   20%|████  

2mTraining trees...                   63%|███████████     |  ETA: 0:01:32
2mTraining trees...                   65%|███████████     |  ETA: 0:01:27
2mTraining trees...                   67%|███████████     |  ETA: 0:01:23
2mTraining trees...                   68%|███████████     |  ETA: 0:01:19
2mTraining trees...                   70%|████████████    |  ETA: 0:01:15
2mTraining trees...                   72%|████████████    |  ETA: 0:01:10
2mTraining trees...                   73%|████████████    |  ETA: 0:01:06
2mTraining trees...                   75%|█████████████   |  ETA: 0:01:02
2mTraining trees...                   77%|█████████████   |  ETA: 0:00:59
2mTraining trees...                   78%|█████████████   |  ETA: 0:00:54
2mTraining trees...                   80%|█████████████   |  ETA: 0:00:50
2mTraining trees...                   82%|██████████████  |  ETA: 0:00:46
2mTraining trees...                   83%|██████████████  |  ETA: 0:00:42
2mTraining trees...                   

2mTraining trees...                   27%|█████           |  ETA: 0:02:49
2mTraining trees...                   28%|█████           |  ETA: 0:02:44
2mTraining trees...                   30%|█████           |  ETA: 0:02:39
2mTraining trees...                   32%|██████          |  ETA: 0:02:38
2mTraining trees...                   33%|██████          |  ETA: 0:02:34
2mTraining trees...                   35%|██████          |  ETA: 0:02:28
2mTraining trees...                   37%|██████          |  ETA: 0:02:24
2mTraining trees...                   38%|███████         |  ETA: 0:02:21
2mTraining trees...                   40%|███████         |  ETA: 0:02:17
2mTraining trees...                   42%|███████         |  ETA: 0:02:14
2mTraining trees...                   43%|███████         |  ETA: 0:02:10
2mTraining trees...                   45%|████████        |  ETA: 0:02:07
2mTraining trees...                   47%|████████        |  ETA: 0:02:03
2mTraining trees...                   

2mTraining trees...                   90%|███████████████ |  ETA: 0:00:38
2mTraining trees...                   92%|███████████████ |  ETA: 0:00:32
2mTraining trees...                   93%|███████████████ |  ETA: 0:00:25
2mTraining trees...                   95%|████████████████|  ETA: 0:00:19
2mTraining trees...                   97%|████████████████|  ETA: 0:00:12
2mTraining trees...                   98%|████████████████|  ETA: 0:00:06
Refitting with best parameters...    0%|█               |  ETA: N/A
4m  Parameters:  cp=>0.0001082 max_depth=>10

...

2mTraining trees...                   65%|███████████     |  ETA: 0:01:13
2mTraining trees...                   67%|███████████     |  ETA: 0:01:10
2mTraining trees...                   68%|███████████     |  ETA: 0:01:07
2mTraining trees...                   70%|████████████    |  ETA: 0:01:03
2mTraining trees...                   72%|████████████    |  ETA: 0:00:59
2mTraining trees...                   73%|████████████    |  ETA: 0:00:56
2mTraining trees...                   75%|█████████████   |  ETA: 0:00:52
2mTraining trees...                   77%|█████████████   |  ETA: 0:00:49
2mTraining trees...                   78%|█████████████   |  ETA: 0:00:46
2mTraining trees...                   80%|█████████████   |  ETA: 0:00:42
2mTraining trees...                   82%|██████████████  |  ETA: 0:00:38
2mTraining trees...                   83%|██████████████  |  ETA: 0:00:35
2mTraining trees...                   85%|██████████████  |  ETA: 0:00:31
2mTraining trees...                   

2mTraining trees...                   28%|█████           |  ETA: 0:02:30
2mTraining trees...                   30%|█████           |  ETA: 0:02:25
2mTraining trees...                   32%|██████          |  ETA: 0:02:22
2mTraining trees...                   33%|██████          |  ETA: 0:02:17
2mTraining trees...                   35%|██████          |  ETA: 0:02:12
2mTraining trees...                   37%|██████          |  ETA: 0:02:08
2mTraining trees...                   38%|███████         |  ETA: 0:02:03
2mTraining trees...                   40%|███████         |  ETA: 0:02:01
2mTraining trees...                   42%|███████         |  ETA: 0:01:59
2mTraining trees...                   43%|███████         |  ETA: 0:01:57
2mTraining trees...                   45%|████████        |  ETA: 0:01:53
2mTraining trees...                   47%|████████        |  ETA: 0:01:49
2mTraining trees...                   48%|████████        |  ETA: 0:01:45
2mTraining trees...                   

2mTraining trees...                   92%|███████████████ |  ETA: 0:00:19
2mTraining trees...                   93%|███████████████ |  ETA: 0:00:15
2mTraining trees...                   95%|████████████████|  ETA: 0:00:11
2mTraining trees...                   97%|████████████████|  ETA: 0:00:07
2mTraining trees...                   98%|████████████████|  ETA: 0:00:04
Refitting with best parameters...    0%|█               |  ETA: N/A
4m  Parameters:  cp=>0.00005444 max_depth=>10



2mTraining trees...                    0%|█               |  ETA: N/A

2mTraining trees...                    2%|█               |  ETA: 0:04:54

2mTraining trees...                    3%|█               |  ETA: 0:06:46

2mTraining trees...                    5%|█               |  ETA: 0:10:08

2mTraining trees...                    7%|██              |  ETA: 0:09:54

2mTraining trees...                    8%|██              |  ETA: 0:10:46

2mTraining trees...                   10%|██              |  ETA: 0:10:48

2mTraining trees...                   12%|██              |  ETA: 0:11:35

2mTraining trees...                   13%|███             |  ETA: 0:11:58

2mTraining trees...                   15%|███             |  ETA: 0:11:50

2mTraining trees...                   17%|███             |  ETA: 0:11:30

2mTraining trees...                   18%|███             |  ETA: 0:11:23

2mTraining trees...                   20%|████            |  ETA: 0:10:59

2mTraining trees...        




0.8204571537763894
0.7917849264522634
Ending run number 10




## Updated labels

In [32]:
updated_aucs_train = []
updated_aucs_test = []
for num in 1:10
    println("Beginning run number ", num)
    println("\n")
    train_auc, test_auc, best_OCT = pre_process_and_train(weighted_sev, updated_label, num)
    updated_aucs_train = append!(updated_aucs_train, train_auc)
    updated_aucs_test = append!(updated_aucs_test, test_auc)
    IAI.write_json("Fairness_OCT_updated_label $num.json",best_OCT)
    println("Ending run number ", num)
    println("\n")
end

Beginning run number 1


Transformed column 'Head_severity' to Ordered Categorical
Transformed column 'Face_severity' to Ordered Categorical
Transformed column 'Neck_severity' to Ordered Categorical
Transformed column 'Thorax_severity' to Ordered Categorical
Transformed column 'Abdomen_severity' to Ordered Categorical
Transformed column 'Spine_severity' to Ordered Categorical
Transformed column 'Upper_Extremity_severity' to Ordered Categorical
Transformed column 'Lower_Extremity_severity' to Ordered Categorical
Transformed column 'Pelvis_Perineum_severity' to Ordered Categorical
Transformed column 'External_severity' to Ordered Categorical
Transformed column 'gcstot1' to Ordered Categorical
Transformed all other non-ordered Categorical Variables


CV fold...                           0%|█               |  ETA: N/A
2mTraining trees...                    0%|█               |  ETA: N/A
2mTraining trees...                    2%|█               |  ETA: 0:12:15
2mTraining trees...                    3%|█               |  ETA: 0:11:14
2mTraining trees...                    5%|█               |  ETA: 0:10:27
2mTraining trees...                    7%|██              |  ETA: 0:09:40
2mTraining trees...                    8%|██              |  ETA: 0:09:40
2mTraining trees...                   10%|██              |  ETA: 0:09:24
2mTraining trees...                   12%|██              |  ETA: 0:09:19
2mTraining trees...                   13%|███             |  ETA: 0:09:02
2mTraining trees...                   15%|███             |  ETA: 0:08:43
2mTraining trees...                   17%|███             |  ETA: 0:08:26
2mTraining trees...                   18%|███             |  ETA: 0:08:16
2mTraining trees...                   20%|████  

2mTraining trees...                   63%|███████████     |  ETA: 0:02:53
2mTraining trees...                   65%|███████████     |  ETA: 0:02:47
2mTraining trees...                   67%|███████████     |  ETA: 0:02:40
2mTraining trees...                   68%|███████████     |  ETA: 0:02:35
2mTraining trees...                   70%|████████████    |  ETA: 0:02:27
2mTraining trees...                   72%|████████████    |  ETA: 0:02:19
2mTraining trees...                   73%|████████████    |  ETA: 0:02:12
2mTraining trees...                   75%|█████████████   |  ETA: 0:02:05
2mTraining trees...                   77%|█████████████   |  ETA: 0:01:57
2mTraining trees...                   78%|█████████████   |  ETA: 0:01:50
2mTraining trees...                   80%|█████████████   |  ETA: 0:01:42
2mTraining trees...                   82%|██████████████  |  ETA: 0:01:34
2mTraining trees...                   83%|██████████████  |  ETA: 0:01:26
2mTraining trees...                   

2mTraining trees...                   27%|█████           |  ETA: 0:05:25
2mTraining trees...                   28%|█████           |  ETA: 0:05:12
2mTraining trees...                   30%|█████           |  ETA: 0:04:59
2mTraining trees...                   32%|██████          |  ETA: 0:04:52
2mTraining trees...                   33%|██████          |  ETA: 0:04:42
2mTraining trees...                   35%|██████          |  ETA: 0:04:31
2mTraining trees...                   37%|██████          |  ETA: 0:04:22
2mTraining trees...                   38%|███████         |  ETA: 0:04:16
2mTraining trees...                   40%|███████         |  ETA: 0:04:10
2mTraining trees...                   42%|███████         |  ETA: 0:04:03
2mTraining trees...                   43%|███████         |  ETA: 0:03:57
2mTraining trees...                   45%|████████        |  ETA: 0:03:49
2mTraining trees...                   47%|████████        |  ETA: 0:03:43
2mTraining trees...                   

2mTraining trees...                   90%|███████████████ |  ETA: 0:00:49
2mTraining trees...                   92%|███████████████ |  ETA: 0:00:41
2mTraining trees...                   93%|███████████████ |  ETA: 0:00:33
2mTraining trees...                   95%|████████████████|  ETA: 0:00:24
2mTraining trees...                   97%|████████████████|  ETA: 0:00:16
2mTraining trees...                   98%|████████████████|  ETA: 0:00:08
Refitting with best parameters...    0%|█               |  ETA: N/A
4m  Parameters:  cp=>0.00004284 max_depth=>10

...

2mTraining trees...                   65%|███████████     |  ETA: 0:03:19
2mTraining trees...                   67%|███████████     |  ETA: 0:03:10
2mTraining trees...                   68%|███████████     |  ETA: 0:03:01
2mTraining trees...                   70%|████████████    |  ETA: 0:02:52
2mTraining trees...                   72%|████████████    |  ETA: 0:02:43
2mTraining trees...                   73%|████████████    |  ETA: 0:02:34
2mTraining trees...                   75%|█████████████   |  ETA: 0:02:24
2mTraining trees...                   77%|█████████████   |  ETA: 0:02:15
2mTraining trees...                   78%|█████████████   |  ETA: 0:02:05
2mTraining trees...                   80%|█████████████   |  ETA: 0:01:56
2mTraining trees...                   82%|██████████████  |  ETA: 0:01:46
2mTraining trees...                   83%|██████████████  |  ETA: 0:01:36
2mTraining trees...                   85%|██████████████  |  ETA: 0:01:26
2mTraining trees...                   

2mTraining trees...                   28%|█████           |  ETA: 0:07:13
2mTraining trees...                   30%|█████           |  ETA: 0:06:58
2mTraining trees...                   32%|██████          |  ETA: 0:06:48
2mTraining trees...                   33%|██████          |  ETA: 0:06:37
2mTraining trees...                   35%|██████          |  ETA: 0:06:29
2mTraining trees...                   37%|██████          |  ETA: 0:06:16
2mTraining trees...                   38%|███████         |  ETA: 0:06:06
2mTraining trees...                   40%|███████         |  ETA: 0:05:56
2mTraining trees...                   42%|███████         |  ETA: 0:05:49
2mTraining trees...                   43%|███████         |  ETA: 0:05:39
2mTraining trees...                   45%|████████        |  ETA: 0:05:30
2mTraining trees...                   47%|████████        |  ETA: 0:05:18
2mTraining trees...                   48%|████████        |  ETA: 0:05:07
2mTraining trees...                   

2mTraining trees...                   92%|███████████████ |  ETA: 0:00:48
2mTraining trees...                   93%|███████████████ |  ETA: 0:00:38
2mTraining trees...                   95%|████████████████|  ETA: 0:00:29
2mTraining trees...                   97%|████████████████|  ETA: 0:00:19
2mTraining trees...                   98%|████████████████|  ETA: 0:00:10
Refitting with best parameters...    0%|█               |  ETA: N/A
4m  Parameters:  cp=>0.00005812 max_depth=>10



2mTraining trees...                    0%|█               |  ETA: N/A

2mTraining trees...                    2%|█               |  ETA: 0:14:20

2mTraining trees...                    3%|█               |  ETA: 0:12:20

2mTraining trees...                    5%|█               |  ETA: 0:12:54

2mTraining trees...                    7%|██              |  ETA: 0:12:27

2mTraining trees...                    8%|██              |  ETA: 0:12:25

2mTraining trees...                   10%|██              |  ETA: 0:11:44

2mTraining trees...                   12%|██              |  ETA: 0:11:50

2mTraining trees...                   13%|███             |  ETA: 0:11:50

2mTraining trees...                   15%|███             |  ETA: 0:11:28

2mTraining trees...                   17%|███             |  ETA: 0:10:59

2mTraining trees...                   18%|███             |  ETA: 0:10:54

2mTraining trees...                   20%|████            |  ETA: 0:10:34

2mTraining trees...        




0.8904358879954165
0.8774759638131494
Ending run number 10




In [34]:
results = DataFrame(
    aucs_original_train = orig_aucs_train,
    aucs_original_test = orig_aucs_test,
    aucs_updated_train = updated_aucs_train,
    aucs_updated_test = updated_aucs_test,
)

,aucs_original_train,aucs_original_test,aucs_updated_train,aucs_updated_test
,Any,Any,Any,Any
1,0.823936,0.782748,0.894281,0.873143
2,0.823015,0.786437,0.892898,0.86747
3,0.822078,0.789359,0.891893,0.869385
4,0.821097,0.79453,0.893406,0.866078
5,0.822715,0.783774,0.89678,0.859023
6,0.819974,0.792835,0.88873,0.875294
7,0.819728,0.791336,0.892421,0.869406
8,0.820152,0.791517,0.891925,0.86779
9,0.81934,0.785492,0.890814,0.864337


In [36]:
CSV.write("C:/Users/15125/Desktop/CAPSTONE PROJECT/CLEANED DATA/FAIRNESS MODEL/fairness_oct_split_results_depth10.csv", results)

"C:/Users/15125/Desktop/CAPSTONE PROJECT/CLEANED DATA/FAIRNESS MODEL/fairness_oct_split_results_depth10.csv"